In [ ]:
import random
import math
import copy
import numpy as np
from tensorflow.keras.utils import Sequence
from dataset.spatial_transforms import RandomCrop, Scale, RandomScale, RandomHorizontalFlip, CenterCrop, Compose, Normalize
from dataset.tempora_transforms import TemporalRandomCrop, TemporalCenterCrop
#from dataset.utils import load_value_file, load_clip_video

In [ ]:
class DataGenerator(Sequence):
  def __init__(self, video_path, file_path, name_path, 
             mode, batch_size, num_classes, shuffle, 
             short_size = (256,320), crop_size = 224, clip_len=64, n_samples_for_each_video = 1):
    
    if mode == 'train':
      self.spatial_transforms = Compose([
                                         RandomScale(short_side),
                                         RandomCrop(crop_size),
                                         RandomHorizontalFlip(),
                                         Normalize()
      ])
      self.temporal_transforms = TemporalRandomCrop(clip_len)

    elif mode == 'val':
      self.spatial_transforms = Compose([
                                         Scale(short_side),
                                         CenterCrop(crop_size),
                                         Normalize()
      ])
      self.temporal_transforms = TemporalRandomCrop(clip_len)

    else:
      raise ValueError("invalid mode")

    self.dataset = self.makedataset(n_samples_for_each_video, clip_len)
    if self.shuffle:
      random.shuffle(self.dataset)


    def __len__(self):
      return math.ceil(len(self.video_files)/self.batch_size)

    def __getitem__(self, index):
      batch_dataset = self.dataset[index*self.batch_size:(index+1)*self.batch_size]
      video_data, label_data = self.data_generator(batch_dataset)
      return video_data, label_data

    def on_epoch_end(self):
      if self.shuffle:
        random.shuffle(self.dataset)

  def makedataset(self, n_samples_for_each_video, clip_len):
    dataset = []
    for i, video_file in enumrate(self.video_files):
      if i%100==0:
        print('dataset loading [ {0} / {1}]'.format(1, len*self.video_files))
      if not os.path.exists(video_files):
        print('{0} is not exist'.format(video_file))
        continue

      n_frame_path = os.path.join(video_file, 'n_frames')
      n_frames = int(load_value_file(n_frame_path))

      if n_frames<=0:
        continue

      sample = {
          'video_path' : video_file,
          'label' : int(self.label_files[i])
      }
      if n_samples_for_each_video==1:
        sample['frame_indices'] = list(range(1, n_frames+1))
      else:
        if n_samples_for_each_video>1:
          step = max(1, math.ceil((n_frames-1-clip_len)/ (n_samples_for_each_video-1)))
        else:
          step = clip_len

        for j in range(1, n_frames, step):
          sample_j = copy.deepcopy(sample)
          sample_j['frame_indices'] = list(range(j, min(n_frames + 1, j + clip_len)))
          dataset.append(sample_j)
    
    return dataset

    def data_generator(self, batch_dataset):
      video_data = []
      label_data = []
      for data in batch_dataset:    #batch_dataset은 sample의 집합체
        path = data['video_path']
        frame_indeices = data['frame_indices']
        if self.temporal_transforms is not None:
          frame_indices = self.temporal_transforms(frame_indices)
        
        clip = load_clip_video(path, frame_indeices)

        if self.spatial_transforms is not None:
          self.spatial_transforms.randomize_parameters()
          clip = [self.spatial_transforms(img) for img in clip]

        clip = np.stack(clip,0)
        video_data.append(clip)
        label_data.append(data['label'])
      video_data = np.array(video_data)
      label_data = np.eye(self.num_classes)[label_data]

      return video_data, label_data


In [ ]:
n_frames = 128
clip_len=64
n_samples_for_each_video=3
step = max(1, math.ceil((n_frames-1-clip_len)/ (n_samples_for_each_video-1)))
print(step)
dataset = []
video_files = os.listdir('/content/drive/MyDrive/종합설계/movies/NonViolence')
for i, video_file in enumerate(video_files):
  sample = {
                  'video_path':video_file,
                  'label': 'Nonviolence'
              }
  for j in range(1, n_frames, step):
    sample_j = copy.deepcopy(sample)
    sample_j['frame_indices'] = list(range(j, min(n_frames+1, j+clip_len)))
    dataset.append(sample_j)

In [ ]:
]